# IMPORTS

Model imports

In [1]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

KFold cross validation imports

In [2]:
import splitfolders
from sklearn.model_selection import KFold

c:\Users\maxba\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Device

In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# KFold cross validation

In [4]:
splitfolders.ratio("../Data", output="../DataTrainTest", seed=1337, ratio=(.8, 0, .2), group_prefix=None, move=False)

Copying files: 2465 files [00:09, 257.46 files/s]


In [5]:
data_dir = '../Data'

In [11]:
kf = KFold(n_splits=3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [13]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        for train_index, test_index in kf.split(image):
            print(train_index, test_index)

[14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37
 38 39] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 27 28 29 30 31 32 33 34 35 36
 37 38 39] [14 15 16 17 18 19 20 21 22 23 24 25 26]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26] [27 28 29 30 31 32 33 34 35 36 37 38 39]
[14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37
 38 39] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 27 28 29 30 31 32 33 34 35 36
 37 38 39] [14 15 16 17 18 19 20 21 22 23 24 25 26]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26] [27 28 29 30 31 32 33 34 35 36 37 38 39]
[14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37
 38 39] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 27 28 29 30 31 32 33 34 35 36
 37 38 39] [14 15 16 17 18 19 20 21 22 23 24 25 26]


In [14]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

# Image transformation

In [9]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

# Data loader

In [10]:
train_path='../DataTrainTestVal/train/'
test_path='../DataTrainTestVal/test/'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

# Determine classes

In [16]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [17]:
print(classes)

['altocumulus', 'altostratus', 'cirrocumulus', 'cirrostratus', 'cirrus', 'clearSky', 'cumulonimbus', 'cumulus', 'nimbostratus', 'stratocumulus', 'stratus']


# CNN Model

In [18]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=11):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            
        


In [19]:
model=ConvNet(num_classes=11).to(device)

# Optimizer

In [20]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [21]:
num_epochs=10

In [22]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [23]:
print(train_count,test_count)

1829 234


# Training

In [24]:
best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(33.0004) Train Accuracy: 0.1864406779661017 Test Accuracy: 0.17094017094017094
Epoch: 1 Train Loss: tensor(9.0926) Train Accuracy: 0.29852378348824493 Test Accuracy: 0.20085470085470086
Epoch: 2 Train Loss: tensor(4.0929) Train Accuracy: 0.45325314379442316 Test Accuracy: 0.2863247863247863
Epoch: 3 Train Loss: tensor(2.0934) Train Accuracy: 0.5844723892837617 Test Accuracy: 0.41025641025641024
Epoch: 4 Train Loss: tensor(1.5510) Train Accuracy: 0.6965554948059048 Test Accuracy: 0.4188034188034188
Epoch: 5 Train Loss: tensor(1.2204) Train Accuracy: 0.7676325861126299 Test Accuracy: 0.4358974358974359
Epoch: 6 Train Loss: tensor(1.0477) Train Accuracy: 0.8146528157463094 Test Accuracy: 0.44017094017094016
Epoch: 7 Train Loss: tensor(0.8534) Train Accuracy: 0.8589393110989612 Test Accuracy: 0.42735042735042733
Epoch: 8 Train Loss: tensor(0.7778) Train Accuracy: 0.8840896664844177 Test Accuracy: 0.44017094017094016
Epoch: 9 Train Loss: tensor(0.6613) Train Accu